In [ ]:
#https://github.com/rybread1/trump-speech-writer2

###Initialize variables

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import datetime
import itertools
from collections import Counter
import re
import os
import pandas as pd

!pip install transformers -q

     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 45.9 MB/s 
     |████████████████████████████████| 163 kB 42.7 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [148]:
lang = 'no'

In [149]:
df_orig = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final/' + lang + '/data/data_users_as_labels-sent_clean_' + lang + '.csv')

In [150]:
df_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39624 entries, 0 to 39623
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         39624 non-null  int64 
 1   text               39624 non-null  object
 2   user               39624 non-null  object
 3   folder             39624 non-null  object
 4   lang               39624 non-null  object
 5   length             39624 non-null  int64 
 6   translation        39624 non-null  object
 7   translation_clean  39617 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.4+ MB


In [151]:
df_orig.head()

,Unnamed: 0,text,user,folder,lang,length,translation,translation_clean
0,0,\n I don't know if you got these.\n ----------...,bass-e,sent,no,545,Jeg vet ikke om du har disse.\n --------------...,vite $. ---------------------- videresendt eri...
1,1,\n Can you spell S-N-O-O-T-Y?\n \n e\n \n \n ...,bass-e,sent,no,460,Kan du stave S-N-O-O-T-Y?\n \n e\n \n \n \n...,stave s-n-o-o-t-y $? e $: ami chokshi enron 01...
2,2,"\n The new deal #s are 147888,147889.\n \n -E",bass-e,sent,no,42,"De nye avtalenummerene er 147888,147889.\n \n -E","ny avtalenummer 147888,147889. -e"
3,3,\n I think I sent this to the worng e-mail add...,bass-e,sent,no,167,Jeg tror jeg sendte dette til den slitte e-pos...,tro sende slite e-postadresse første gang $( s...
4,4,\n We are going to Woody's tonight if you are ...,bass-e,sent,no,64,Vi skal til Woody's i kveld hvis du er interes...,woody's kveld interessere $. $-e


In [152]:
df = df_orig[['translation']].sample(100)

In [153]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 816 to 8247
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   translation  100 non-null    object
dtypes: object(1)
memory usage: 1.6+ KB


###Initialize custom tokenizer

In [154]:
train_path = '/content/drive/MyDrive/Colab Notebooks/final/' + lang + '/generate/train_dataset_' + lang + '.txt'
test_path = '/content/drive/MyDrive/Colab Notebooks/final/' + lang + '/generate/test_dataset_' + lang + '.txt'

In [155]:
from sklearn.model_selection import train_test_split

def build_text_files(txts, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for txt in txts:
        summary = txt.strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

train, test = train_test_split(df.translation, test_size=0.15)

build_text_files(train, train_path)
build_text_files(test, test_path)

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 85
Test dataset length: 15


In [156]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/909a290700bd99135e67c64eefc166960b67cfd2/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/909a290700bd99135e67c64eefc166960b67cfd2/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/909a290700bd99135e67c64eefc166960b67cfd2/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12

###Train custom tokenizer

In [157]:
dataset = df.translation.tolist()

In [158]:
dataset[3]

'Ingen stor sak. Jeg prøver bare å binde opp løse tråder når jeg kan. Jeg mistenker\n de må ha fått betalt, ellers ville noen klaget.'

In [159]:
batch_size = 1000
def batch_iterator():
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]

In [160]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/909a290700bd99135e67c64eefc166960b67cfd2/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-g

In [161]:
tokenizer_name = 'gpt2-tokenizer_' + lang

In [162]:
%%time
new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=25000)

CPU times: user 738 ms, sys: 161 ms, total: 899 ms
Wall time: 863 ms


In [163]:
new_tokenizer.save_pretrained('/content/drive/MyDrive/Colab Notebooks/final/' + lang + '/generate/'+tokenizer_name)

tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/final/no/generate/gpt2-tokenizer_no/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/final/no/generate/gpt2-tokenizer_no/special_tokens_map.json


('/content/drive/MyDrive/Colab Notebooks/final/no/generate/gpt2-tokenizer_no/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/final/no/generate/gpt2-tokenizer_no/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/final/no/generate/gpt2-tokenizer_no/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/final/no/generate/gpt2-tokenizer_no/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/final/no/generate/gpt2-tokenizer_no/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/final/no/generate/gpt2-tokenizer_no/tokenizer.json')

In [164]:
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Colab Notebooks/final/' + lang + '/generate/' + tokenizer_name)

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


###Train model

In [165]:
%%time
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):

    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=16)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=16)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,

    )
    return train_dataset,test_dataset,data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Creating features from dataset file at /content/drive/MyDrive/Colab Notebooks/final/no/generate
Token indices sequence length is longer than the specified maximum sequence length for this model (37901 > 1024). Running this sequence through the model will result in indexing errors
Saving features into cached file /content/drive/MyDrive/Colab Notebooks/final/no/generate/cached_lm_GPT2TokenizerFast_16_train_dataset_no.txt [took 0.008 s]
Creating features from dataset file at /content/drive/MyDrive/Colab Notebooks/final/no/generate
Saving features into cached file /content/drive/MyDrive/Colab Notebooks/f

CPU times: user 344 ms, sys: 12.8 ms, total: 357 ms
Wall time: 471 ms


In [166]:
len(train_dataset.examples)

2368

In [167]:
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained("gpt2")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/909a290700bd99135e67c64eefc166960b67cfd2/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.22.2",
  "use_cach

In [168]:
%%time
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Colab Notebooks/final/' + lang + '/generate/gpt2_' + lang, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)
    #prediction_loss_only=True,

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


CPU times: user 13.2 ms, sys: 3.01 ms, total: 16.2 ms
Wall time: 37.8 ms


###Train model fo real

In [169]:
%%time
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2368
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 222


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 29min 11s, sys: 37.8 s, total: 29min 49s
Wall time: 29min 48s


TrainOutput(global_step=222, training_loss=8.568015708579674, metrics={'train_runtime': 1788.7371, 'train_samples_per_second': 3.972, 'train_steps_per_second': 0.124, 'total_flos': 58006831104000.0, 'train_loss': 8.568015708579674, 'epoch': 3.0})

In [170]:
trainer.save_model('/content/drive/MyDrive/Colab Notebooks/final/' + lang + '/generate/')

Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/final/no/generate/
Configuration saved in /content/drive/MyDrive/Colab Notebooks/final/no/generate/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/final/no/generate/pytorch_model.bin


In [171]:
model_generate = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/Colab Notebooks/final/' + lang + '/generate/')

loading configuration file /content/drive/MyDrive/Colab Notebooks/final/no/generate/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.22.2",
  "use_cache": tru

In [ ]:
from transformers import pipeline

generate = pipeline('text-generation', model=model_generate, tokenizer='gpt2', config={'max_length':10000, 'temperature': .5})

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/909a290700bd99135e67c64eefc166960b67cfd2/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-g

In [ ]:
initial_seed = df.sample(1).values[0][0]

In [ ]:
initial_seed

In [ ]:
%%time
text = generate(initial_seed, max_length=250, min_length=100, do_sample=True, temperature=1.2)[0]['generated_text']

In [ ]:
text